In [1]:
import numpy as np
from scipy.optimize import minimize


- https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model
- https://chatgpt.com/c/672eeaa4-4b84-800e-832b-87dad7fa7f66

In [5]:
import numpy as np
from scipy.optimize import minimize

# 胜利次数矩阵
w = np.array([
    [0, 2, 0, 1],
    [3, 0, 5, 0],
    [0, 3, 0, 1],
    [4, 0, 3, 0]
])

# 总比赛次数矩阵
n = w + w.T

def neg_log_likelihood(theta):
    theta_full = np.concatenate(([0], theta))
    exp_theta = np.exp(theta_full)
    L = 0
    for i in range(len(w)):
        for j in range(i+1, len(w)):
            if n[i, j] > 0:
                L += (
                    w[i, j] * theta_full[i] +
                    w[j, i] * theta_full[j] -
                    n[i, j] * np.log(exp_theta[i] + exp_theta[j])
                )
    return -L

# 设置初始猜测和收敛条件
theta_opt = np.zeros(3)
tolerance = 1e-6
max_iterations = 10

for iteration in range(max_iterations):
    result = minimize(
        neg_log_likelihood,
        theta_opt,
        method='BFGS'
    )
    
    if not result.success:
        raise Exception('Optimization failed: ' + result.message)
    
    # 检查参数变化是否小于阈值
    if np.all(np.abs(theta_opt - result.x) < tolerance):
        print(f"Optimization converged after {iteration + 1} iterations.")
        break
    
    theta_opt = result.x

theta_full = np.concatenate(([0], theta_opt))
p = np.exp(theta_full)
p_normalized = p / np.sum(p)

# 输出结果
teams = ['A', 'B', 'C', 'D']
print("团队强度参数 p_i：")
for i in range(len(teams)):
    print(f"团队 {teams[i]} 的强度参数 p_{teams[i]} = {p[i]:.4f}")

print("\n归一化的团队强度参数 p_i (总和为1)：")
for i in range(len(teams)):
    print(f"团队 {teams[i]} 的归一化强度参数 p_{teams[i]} = {p_normalized[i]:.4f}")


Optimization converged after 2 iterations.
团队强度参数 p_i：
团队 A 的强度参数 p_A = 1.0000
团队 B 的强度参数 p_B = 1.6306
团队 C 的强度参数 p_C = 1.0312
团队 D 的强度参数 p_D = 3.5484

归一化的团队强度参数 p_i (总和为1)：
团队 A 的归一化强度参数 p_A = 0.1387
团队 B 的归一化强度参数 p_B = 0.2262
团队 C 的归一化强度参数 p_C = 0.1430
团队 D 的归一化强度参数 p_D = 0.4921


- Paired Comparison Analysis
    
    Suppose 100 consumers compare five tyre brands as follows:
    
    | Brand   | Dunlop | Modi | Ceat | GoodYear | MRF |
    |---------|--------|------|------|----------|-----|
    | Dunlop  | -      | 80   | 59   | 52       | 77  |
    | Modi    | 20     | -    | 60   | 46       | 56  |
    | Ceat    | 41     | 40   | -    | 61       | 60  |
    | G.Year  | 48     | 54   | 39   | -        | 67  |
    | MRF     | 23     | 44   | 40   | 33       | -   |

- 我们引入一个正的能力参数 $\pi_i\gt 0$，表示消费者对品牌 $i$ 的偏好强度。
- 对于每一对不同的品牌 $(i,j)$，我们有以下的观测数据
    - $n_{ij}$, 在比较中消费者选择品牌 $i$ 而不是品牌 $j$ 的次数；
    - $n_{ji}$, 在比较中消费者选择品牌 $j$ 而不是品牌 $i$ 的次数；
    - $n_{ij}+n_{ji}=n_{ij}^{total}$，两品牌被比较的总次数；
- Bradley-Terry 模型的概率表示
    - 假设品牌 $i$ 被偏好于品牌 $j$ 的概率 $P{ij}$
        - $P_{ij}=\frac{\pi_i}{\pi_{i}+\pi_j}$
    - 相应地
        - $P_{ji}=\frac{\pi_j}{\pi_i+\pi_j}$
- 基于成对比较数据和 Bradley-Terry 模型，我们可以构建对数似然函数
    - $\ell(\pi)=\sum n_{ij}\log\frac{\pi_i}{\pi_i+\pi_j}+n_{ji}\log\frac{\pi_j}{\pi_i+\pi_j}$


In [2]:

# Brands and indices
brands = ['Dunlop', 'Modi', 'Ceat', 'GoodYear', 'MRF']
brand_indices = {brand: idx for idx, brand in enumerate(brands)}

# Paired comparison data: (i, j, n_ij, n_ji)
data = [
    (0, 1, 80, 20),  # Dunlop vs Modi
    (0, 2, 59, 41),  # Dunlop vs Ceat
    (0, 3, 52, 48),  # Dunlop vs GoodYear
    (0, 4, 77, 23),  # Dunlop vs MRF
    (1, 2, 60, 40),  # Modi vs Ceat
    (1, 3, 46, 54),  # Modi vs GoodYear
    (1, 4, 56, 44),  # Modi vs MRF
    (2, 3, 61, 39),  # Ceat vs GoodYear
    (2, 4, 60, 40),  # Ceat vs MRF
    (3, 4, 67, 33)   # GoodYear vs MRF
]

def neg_log_likelihood(log_pis):
    # log_pis is an array of length 4 (log_p0 to log_p3)
    # log_p4 is fixed at 0
    log_pis_full = np.append(log_pis, 0)  # Append log_p4 = 0
    LL = 0
    for i, j, n_ij, n_ji in data:
        log_pi = log_pis_full[i]
        log_pj = log_pis_full[j]
        max_log_pi_pj = max(log_pi, log_pj)
        # Compute log_sum_exp(log_pi, log_pj) in a numerically stable way
        log_sum_ij = max_log_pi_pj + np.log( np.exp(log_pi - max_log_pi_pj) + np.exp(log_pj - max_log_pi_pj) )
        LL += n_ij * (log_pi - log_sum_ij) + n_ji * (log_pj - log_sum_ij)
    return -LL  # Negative log-likelihood


In [3]:
# Initial guesses for log_p0 to log_p3
x0 = np.zeros(4)

# Perform optimization
result = minimize(neg_log_likelihood, x0, method='BFGS')

if result.success:
    log_pis_est = np.append(result.x, 0)  # Append log_p4 = 0
    pis_est = np.exp(log_pis_est)
    pis_est_normalized = pis_est / np.sum(pis_est)
    print("Estimated Ability Scores:")
    for idx, brand in enumerate(brands):
        print(f"{brand}: {pis_est_normalized[idx]:.4f}")
else:
    print("Optimization failed:", result.message)

Estimated Ability Scores:
Dunlop: 0.3335
Modi: 0.1618
Ceat: 0.1908
GoodYear: 0.2005
MRF: 0.1134
